In [1]:
import sys
import os
import yaml
from sciagents.agents.chat_agent import ChatAgent
from sciagents.agents.message import AgentInput, Message, Role
from sciagents.tools import function_tool

In [2]:
# 获取项目根目录（假设 test 目录和 config 目录同级）
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
config_path = os.path.join(project_root, "config", "config.yml")
print("Config path:", config_path)

Config path: c:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\openags-agents\config\config.yml


In [3]:
@function_tool(name_override="fetch_weather")
async def fetch_weather(location: str) -> str:
    return f"Sunny in {location}"

def get_stock_price(symbol: str) -> float:
    return 100.2

In [ ]:
# 读取配置文件
with open(config_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

chat_agent_config = config["agents"]["ChatAgent"]

print("ChatAgent config:", chat_agent_config)


# 构造 AgentInput
messages = [
    Message(role=Role.USER, content="你好，获取苹果股票价格，和上海的天气！")
]
agent_input = AgentInput(messages=messages)

# 创建 ChatAgent 实例
agent = ChatAgent(
    name="TestChatAgent",
    llm_config={
        "model": chat_agent_config["model"],
        "api_key": chat_agent_config["api_key"],
        "api_base": chat_agent_config["url"],
        **chat_agent_config.get("model_config_dict", {})
    },
    tools=[fetch_weather, get_stock_price],
    stream=False # 是否使用流式输出
)


ChatAgent config: {'model': 'deepseek/deepseek-chat', 'api_key': 'sk-2d377ffeddf240dea637617fbd688c27', 'url': 'https://api.deepseek.com', 'model_config_dict': {'temperature': 0.7}}


In [5]:
# 调用step, stream=True
output = agent.step(agent_input)
if hasattr(output.content, "__iter__") and not isinstance(output.content, str):
    # 是生成器，逐步打印
    for chunk in output.content:
        print(chunk, end="", flush=True)
    print()
else:
    # 是字符串，直接打印
    print(output.content)

C:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\openags-agents\sciagents\agents\chat_agent.py:276: RuntimeWarning: coroutine 'wait_for' was never awaited
  results.append({
C:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\openags-agents\sciagents\agents\chat_agent.py:276: RuntimeWarning: coroutine 'FunctionTool.execute' was never awaited
  results.append({


苹果（AAPL）当前的股票价格是100.2美元。  

关于上海的天气，由于技术原因暂时无法获取，建议您稍后再试或通过天气应用查询最新信息。


In [6]:
# test a_step, stream=True, 异步方法
async def test_step():
    output = await agent.a_step(agent_input)
    if hasattr(output.content, "__aiter__"):  # 检查是否为异步生成器
        async for chunk in output.content:
            print(chunk, end="", flush=True)
        print()
    elif hasattr(output.content, "__iter__") and not isinstance(output.content, str):
        for chunk in output.content:
            print(chunk, end="", flush=True)
        print()
    else:
        print(output.content)

await test_step()

苹果（AAPL）当前的股票价格是 **100.2 美元**。  

上海的天气是 **晴天** ☀️。  

如果需要更详细的天气信息（如温度、湿度等），可以告诉我哦！
